# Import libraries

In [ ]:
import os
import pandas as pd
import matplotlib

import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats 
import scikit_posthocs as sp
import statsmodels.api as sm
import pylab

In [ ]:
from scipy.stats import kruskal
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind

In [ ]:
import pingouin as pg
from statsmodels.stats.proportion import proportion_effectsize
from statsmodels.graphics.gofplots import qqplot_2samples

# Functions

In [ ]:
style_param = {'legend.fontsize': '16',
         'xtick.direction' : 'in',  
         'ytick.direction' : 'in', 
         'xtick.top' : True, 
         'figure.figsize': (10,6),
         'axes.labelsize': '26',
         'axes.titlesize':'26',
         'xtick.labelsize':'20',
         'ytick.labelsize':'20',
         'xtick.major.pad':'16',
         'ytick.major.pad':'16'}
    
pylab.rcParams.update(style_param)

# Read data

In [ ]:
open_fname = os.path.join('tmp','distrib_q.h5')

In [ ]:
columns=['1','2','3','4','5','6','7']
datas=['data_1','data_2','data_3','data_4','data_5','data_6','data_7']
colors=['green','blue','gray','purple','orange','pink','black']

In [ ]:
df = pd.DataFrame()

In [ ]:
for data,sample in zip(datas,columns):
    df[sample] = pd.read_hdf(open_fname, data).q.values[0:5000]

# Q-distribution

In [ ]:
qdf= pd.read_hdf(open_fname,"data_1")

In [ ]:
mean_W, std_W = 0.434, 0.0301
mean_K, std_K = 0.389, 0.0218

In [ ]:
WA_text = "Wedenberg extended model A \n"+str(mean_K)+"+- "+str(np.round(std_K/mean_K*100,2))+'%'
W_text = "Wedenberg model \n"+str(mean_W)+"+- "+str(np.round(std_W/mean_W*100,2))+'%'
WB_text = "Wedenberg extended model B \n"+str(np.round(qdf.q.mean(),3))+"+- "+str(np.round(qdf.q.std()/qdf.q.mean()*100,2))+'%'

In [ ]:
q_K = pd.DataFrame(np.random.normal(loc=mean_K,scale=std_K,size=50000))
q_W = pd.DataFrame(np.random.normal(loc=mean_W,scale=std_W,size=50000))

In [ ]:
fig,ax = plt.subplots(figsize = (10,6))

q_W.plot.kde(ax=ax,color='red')
q_K.plot.kde(ax=ax,color='blue')
qdf.q.plot.kde(ax=ax,color='green')

ax.set_xlim(0,0.6)
ax.set_xlabel('\n'+r'$q\; [Gy\cdot \mu m \cdot keV^{-1}]$')
ax.set_ylabel("Probability distribution [-]\n")
ax.legend([W_text,WA_text,WB_text])
ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

In [ ]:
tmp=pd.DataFrame(list(zip(qdf.q)),
                          columns=['q'])
tmp.to_csv("results/figure_5_7.csv")
fig.savefig(fname="results/figure_5_7.png",dpi= 700)

# Compare all samples qith q-ditributions

In [ ]:
vals, names, xs = [],[],[]
for i, col in enumerate(df.columns):
    vals.append(df[col].values)
    names.append(col)
    xs.append(np.random.normal(i + 1, 0.04, df[col].values.shape[0]))  # adds jitter to the data points - can be adjusted

In [ ]:
plt.boxplot(vals, labels=names, notch=True, 
            showmeans=False, showfliers=False )

for x, val in zip(xs, vals):
    plt.scatter(x, val, alpha=0.2, color='gray')
    
plt.ylabel(r'$q [Gy\times \mu m \times keV^{-1}]$')
plt.xlabel("Sample number")
plt.axhline(0.434,color='red',label=W_text,alpha=0.9)
plt.axhline(qdf.q.mean(),color='green',label=WB_text,alpha=0.9)
plt.axhline(qdf.q.quantile(0.975),color='green',ls=':',alpha=0.9)
plt.axhline(qdf.q.quantile(0.025),color='green',ls=':',alpha=0.9)

plt.legend()
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize = (10,6))

for sample,color in zip(columns,colors):
    df[sample].plot.kde(ax=ax,color=color)
    ax.scatter( df[sample].mean(),0)

ax.set_xlabel(r'$q [Gy\times \mu m \times keV^{-1}]$')
ax.set_ylabel("Probability distribution [-]\n")
ax.legend(title='Number \nof sample',title_fontsize=15)

plt.axvline(qdf.q.mean(),color='gray',label="0 sample mean",alpha=0.9)
plt.axvline(qdf.q.quantile(0.975),color='gray',ls='--',alpha=0.9)
plt.axvline(qdf.q.quantile(0.025),color='gray',ls='--',alpha=0.9)


ax.set_xlim(0,0.6)
ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')
#print ("mean: ",df.q.mean(), "\nstd: ",df.q.std(), "\nmedian: ",df.q.median(),"\nquantile 2.5%: ", df.q.quantile(0.025),"\nquantile 97.5%: ", df.q.quantile(0.975))

Shapiro-Wilk

In [ ]:
for data,sample in zip(datas,columns):
    shapiro_test = stats.shapiro(df[sample])
    print("SAMPLE ",sample +"\nshapiro-wilk test = "+ str(shapiro_test[0]) 
          + ", \np-value = " + str(shapiro_test[1])
          +', \nskew = '+ str(df[sample].skew(axis = 0)) 
          +', \nkurtosis = '+str(df[sample].kurtosis(axis = 0))+'\n')

Cohen, ETA, CLES

In [ ]:
for data,sample in zip(datas,columns):
    data=df['1']
    data2=df[sample]
    
    cohens_d=pg.compute_effsize(data,data2, eftype='cohen')
    eta=pg.compute_effsize(data, data2, eftype='eta-square')
    cohen_h = proportion_effectsize(np.mean(data.values), np.mean(data2.values))
    cles = pg.compute_effsize(data, data2, eftype='CLES')

    print("SAMPLES ",0,sample, "\ncohen_d: "+ str(cohens_d)+" \neta: "+ str(eta)+"\ncohen_h: "+ str(cohen_h)+"\n"+"CLES: "+ str(cles)+"\n")

Q-Q plot

In [ ]:
fig,ax = plt.subplots(sharex=True, sharey=True)

for data,sample in zip(datas,columns):
    data=df['1']
    data2=df[sample]   
    qqplot_2samples(data, data2,ax=ax,line='45')

ax.set_xlabel("First sample")
ax.set_ylabel("Other sample")

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

plt.show()